In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, EvalPrediction, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Subset 
from torch.optim import AdamW
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import wandb
import optuna
import pandas as pd
import numpy as np

### Save the path to the different datasets

In [3]:
train_en_path = "./data_sources/train/train_en.csv"
test_en_path = "./data_sources/test/test_en.csv"

train_it_path = "./data_sources/train/train_it.csv"
test_it_path = "./data_sources/test/test_it.csv"

train_es_path = "./data_sources/train/train_es.csv"
test_es_path = "./data_sources/test/test_es.csv"

### Set up W&B

In [4]:
wandb.login()

wandb: Currently logged in as: sravisconti (sravisconti-projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Load data in DatasetDict

In [5]:
# Load train and test CSVs
# dataset_it = load_dataset("csv", data_files={
#     "train": train_it_path,
#     "test": test_it_path
# })

# Load CSV manually for the train split
train_df = pd.read_csv(train_it_path)
test_df = pd.read_csv(test_it_path)

# Split into train and val with stratification
train_df, val_df = train_test_split(
    train_df, test_size=0.2, stratify=train_df["label"], random_state=42
)

# Convert back to Hugging Face Datasets
dataset_it = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "val": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df)
})

In [5]:
print(dataset_it)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'bio', 'label', 'lang'],
        num_rows: 694
    })
    val: Dataset({
        features: ['id', 'text', 'bio', 'label', 'lang'],
        num_rows: 174
    })
    test: Dataset({
        features: ['id', 'text', 'bio', 'label', 'lang'],
        num_rows: 218
    })
})


### Tokenize the dataset

In [6]:
model_name = "xlm-roberta-base"
# TODO: later try with "cardiffnlp/twitter-xlm-roberta-base"

# loads the correct tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# define a function to tokenize the text data
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

# applies tokenize function to batches of examples to tokenize the entire dataset
# Returns a dictionary with:
    # input_ids → token IDs (instead of raw text)
    # attention_mask → mask indicating which tokens are real vs padding
    # label → original label from dataset
tokenized_dataset_it = dataset_it.map(tokenize, batched=True)

# the type of the input_ids is list
# batch = tokenized_dataset_it["train"][0]
# print(type(batch["input_ids"]))

# set the format of the dataset to PyTorch tensors before feeding it to the model
tokenized_dataset_it.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

# now the type of the input_ids is torch.Tensor
# batch = tokenized_dataset_it["train"][0]
# print(type(batch["input_ids"]))

Map: 100%|██████████| 218/218 [00:00<00:00, 9096.10 examples/s]


### Define metrics

In [11]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")

    # For ROC AUC, we need probabilities
    if p.predictions.shape[1] == 2:  # binary classification
        probs = torch.softmax(torch.tensor(p.predictions), dim=1)[:,1].numpy()
        roc_auc = roc_auc_score(labels, probs)
    else:
        roc_auc = 0.0

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc
    }

### Define custom Trainer

In [8]:
# Compute class weights from training set
labels = np.array(tokenized_dataset_it["train"]["label"])
print("Labels distribution in training set:", np.bincount(labels))
class_counts = np.bincount(labels)
total = len(labels)
# We want to give more weight to the class that has smaller count
class_weights = total / (len(class_counts) * class_counts)
weights_tensor = torch.tensor(class_weights, dtype=torch.float)

print("Class weights:", weights_tensor)

# Custom Trainer that uses weighted cross entropy
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(weight=weights_tensor.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

Labels distribution in training set: [562 132]
Class weights: tensor([0.6174, 2.6288])


### Define objective function for Optuna

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = 2  # binary classification
batch_size = 8
num_epochs = 4

# Defines one experiment (trial)
def objective(trial):
    # Create a new W&B run for this trial
    wandb.init(
        project="multi-pride-bert-baseline_fixed_epochs_and_batch_size", 
        name=f"trial-{trial.number}",
        reinit="return_previous"
    )

    # Sample hyperparameters for this trial
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    # batch_size = trial.suggest_categorical("batch_size", [8, 16])
    # num_epochs = trial.suggest_int("num_epochs", 2, 5)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    # Initialize new model for this trial
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir=f"./results/trial-{trial.number}",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="steps",   # <-- log training loss every few steps
        logging_steps=10,           # <-- frequency of training loss logging
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        num_train_epochs=num_epochs,
        logging_dir="./logs",
        report_to="wandb",
        run_name=f"trial-{trial.number}",
        load_best_model_at_end=True,
        metric_for_best_model="f1"
    )

    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset_it["train"],
        eval_dataset=tokenized_dataset_it["val"],
        compute_metrics=compute_metrics
    )

    train_result = trainer.train()
    eval_result = trainer.evaluate()

    # ---- 8. Log both train & eval metrics to W&B ----
    wandb.log({
        "train/loss": train_result.training_loss,
        "eval/loss": eval_result["eval_loss"],
        "eval/f1": eval_result["eval_f1"],
        "eval/precision": eval_result["eval_precision"],
        "eval/recall": eval_result["eval_recall"],
        "eval/accuracy": eval_result["eval_accuracy"],
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "weight_decay": weight_decay,
        "num_epochs": num_epochs
    })

    # Finish this run cleanly
    wandb.finish()

    # Return metric Optuna should maximize
    return eval_result["eval_f1"]

### Optuna Tuning

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)  
print("Best trial:")
print(study.best_trial.params)

[I 2025-11-11 15:07:57,659] A new study created in memory with name: no-name-be6ccac4-a48c-43a2-b66f-d3908ac89198


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.478000,0.255537,0.942529,0.914184,0.886485,0.952934,0.965184
2,0.399200,0.187726,0.959770,0.933786,0.939040,0.928756,0.981947
3,0.387900,0.256179,0.965517,0.943907,0.943907,0.943907,0.987750
4,0.353600,0.513945,0.948276,0.905589,0.970000,0.863636,0.977649


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


...

[I 2025-11-11 19:38:53,253] Trial 7 finished with value: 0.9214978569817279 and parameters: {'learning_rate': 2.5134362640363844e-05, 'weight_decay': 0.2588509643495115}. Best is trial 4 with value: 0.9537897700111542.


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.416700,0.191139,0.936782,0.906520,0.877165,0.949387,0.966043
2,0.458300,0.337532,0.959770,0.928567,0.976351,0.893939,0.984956
3,0.228100,0.381831,0.965517,0.939583,0.979592,0.909091,0.990974
4,0.121500,0.407513,0.954023,0.919444,0.957672,0.890393,0.987965


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[W 2025-11-11 20:13:29,538] Trial 8 failed with parameters: {'learning_rate': 1.9651715790283235e-05, 'weight_decay': 0.10665126050623909} because of the following error: RuntimeError('[enforce fail at inline_container.cc:664] . unexpected pos 46720 

RuntimeError: [enforce fail at inline_container.cc:664] . unexpected pos 46720 vs 46612

### Train final model

In [12]:
# Merge train + val for final training
final_train_df = pd.concat([train_df, val_df])
final_train_dataset = Dataset.from_pandas(final_train_df)
final_train_dataset = final_train_dataset.map(tokenize, batched=True)
final_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Load best hyperparameters
# best_params = study.best_trial.params
best_params = {
    "learning_rate": 1.0327292968111504e-05,
    "weight_decay": 0.06144150815589746
}

final_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

final_args = TrainingArguments(
    output_dir="./final_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, 
    logging_strategy="steps",   # <-- log training loss every few steps
    logging_steps=10,           # <-- frequency of training loss logging
    learning_rate=best_params["learning_rate"],
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=best_params["weight_decay"],
    num_train_epochs=num_epochs,
    logging_dir="./logs",
    report_to="wandb",
    run_name="final-model",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

final_trainer = WeightedTrainer(
    model=final_model,
    args=final_args,
    train_dataset=final_train_dataset,
    eval_dataset=tokenized_dataset_it["test"],  # final test only once!
    compute_metrics=compute_metrics,
)

wandb.init(
        project="multi-pride-bert-baseline_fixed_epochs_and_batch_size", 
        name=f"final model",
        reinit="return_previous",
        resume="allow"
    )

final_trainer.train()
final_results = final_trainer.evaluate()
wandb.log(final_results)
print("Final test metrics:", final_results)

Map: 100%|██████████| 868/868 [00:00<00:00, 11521.39 examples/s]
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.449100,0.390409,0.917431,0.867289,0.867289,0.867289,0.950758
2,0.566600,0.283283,0.935780,0.894860,0.902528,0.887716,0.966450
3,0.180300,0.431339,0.935780,0.894860,0.902528,0.887716,0.963609
4,0.323700,0.481175,0.935780,0.894860,0.902528,0.887716,0.966315


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Final test metrics: {'eval_loss': 0.2832830548286438, 'eval_accuracy': 0.9357798165137615, 'eval_f1': 0.8948601350420284, 'eval_precision': 0.9025280898876404, 'eval_recall': 0.8877164502164503, 'eval_roc_auc': 0.9664502164502164, 'eval_runtime': 28.9454, 'eval_samples_per_second': 7.531, 'eval_steps_per_second': 0.967, 'epoch': 4.0}
